In [5]:
import pandas as pd
import pickle
import os
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Пути для сохранения
EMBEDDINGS_FILE = 'recipe_embeddings.pkl'
RECIPES_DATA_FILE = 'recipes_data.pkl'

# 1. Загрузка данных
df = pd.read_csv('cleaned_eda_all_recipes.csv')

# 2. Формируем "вкусовой профиль"
if 'ingridient_keywords' in df.columns:
    df['flavor_profile'] = df['ingridient_keywords'].str.replace(r"[{}']", "", regex=True)
elif 'ingredient_keywords' in df.columns:
    df['flavor_profile'] = df['ingredient_keywords'].str.replace(r"[{}']", "", regex=True)
else:
    df['flavor_profile'] = df['list_ingrid'].str.replace(r'[^\w\s]', '', regex=True)

# 3. Проверяем, есть ли уже сохранённые эмбеддинги
if os.path.exists(EMBEDDINGS_FILE) and os.path.exists(RECIPES_DATA_FILE):
    print("🔁 Загружаем сохранённые эмбеддинги и модель...")
    with open(EMBEDDINGS_FILE, 'rb') as f:
        model, recipe_embeddings = pickle.load(f)
else:
    print("🧠 Обучаем модель и векторизуем рецепты...")
    model = SentenceTransformer('sentence-transformers/distiluse-base-multilingual-cased-v2')
    recipe_embeddings = model.encode(df['flavor_profile'].tolist(), show_progress_bar=True)

    # Сохраняем для следующих запусков
    with open(EMBEDDINGS_FILE, 'wb') as f:
        pickle.dump((model, recipe_embeddings), f)

    # Сохраняем только нужные данные
    df[['name', 'list_ingrid', 'list_resipe']].to_pickle(RECIPES_DATA_FILE)

# 4. Функция поиска
def find_recipes(query, top_n=5):
    query_emb = model.encode([query])
    scores = cosine_similarity(query_emb, recipe_embeddings).flatten()
    idxs = scores.argsort()[-top_n:][::-1]
    return df.iloc[idxs][['name', 'list_ingrid', 'list_resipe']]

# 5. Пример использования
if __name__ == "__main__":
    user_query = input("Введите ваши вкусовые предпочтения: ")
    print("\n🔎 Подбираем рецепты...\n")

    results = find_recipes(user_query)
    for i, row in results.iterrows():
        print(f"🍽️ Название: {row['name']}")
        print(f"🛒 Ингредиенты: {row['list_ingrid']}")
        print(f"📝 Рецепт: {row['list_resipe']}")
        print("-" * 60)

🔁 Загружаем сохранённые эмбеддинги и модель...

🔎 Подбираем рецепты...

🍽️ Название: Вареный краб
🛒 Ингредиенты: Крабы 
📝 Рецепт: 1. Поставьте на огонь огромную кастрюлю с соленой водой. Так быстро, как вы сможете, опустите всех крабов в воду. Удобнее всего варить крабов в сетке, или вы можете воспользоваться щипцами.2. Закройте кастрюлю немедленно. Оставьте крабов вариться на 8 минут.3. Достаньте крабов из кастрюли и поместите в холодильник на несколько часов (если вы предпочитаете не очень холодных) или на всю ночь (если вам нравится холодное мясо крабов в застывшем крабовом соке).
------------------------------------------------------------
🍽️ Название: nan
🛒 Ингредиенты: nan
📝 Рецепт: nan
------------------------------------------------------------
🍽️ Название: Паста из печеного чеснока
🛒 Ингредиенты: Чеснок 2 головки 
📝 Рецепт: 1. Разогрейте духовку до 180 градусов.2. Срежьте у головок небольшие макушки, чтобы слегка стали видны зубчики. заверните каждую в фольгу и переложите на п